In [2]:
#MUST RUN FIRST
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import pygsheets
import time

load_dotenv()

api_key = "RGAPI-a76bbb3e-4659-4021-b035-f0845041d3ed"
print(api_key)

service_account = pygsheets.authorize(service_file='JSONS\spreadsheet-automator-475823-5c8c84bc15e3.json')

sheet = service_account.open_by_url('https://docs.google.com/spreadsheets/d/1rOsmVGBkR-1OcwmnIbIDt-J50JX0O0zHtIGGDKxOzmI/edit?usp=sharing')

def get_puuid(gameName=None, tagline=None, api_key=None):
    link = f'https://europe.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagline}?api_key={api_key}'

    response = requests.get(link)

    return response.json()['puuid']

def get_name_and_tag(puuid=None, api_key=None):
    link = f'https://europe.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}'

    response = requests.get(link)
    
    gameName = response.json()['gameName']
    tagline = response.json()['tagLine']

    return f'{gameName}#{tagline}'

def get_ladder(top=None):

    root = 'https://euw1.api.riotgames.com/'
    chall = 'lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    gm = 'lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
    master = 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'


    chall_response = requests.get(root + chall + f'?api_key={api_key}')
    gm_response = requests.get(root + gm + f'?api_key={api_key}')
    master_response = requests.get(root + master + f'?api_key={api_key}')

    chall_df = pd.DataFrame(chall_response.json()['entries']).sort_values(by='leaguePoints', ascending=False).reset_index(drop=True)
    gm_df = pd.DataFrame(gm_response.json()['entries']).sort_values(by='leaguePoints', ascending=False).reset_index(drop=True)
    master_df = pd.DataFrame(master_response.json()['entries']).sort_values(by='leaguePoints', ascending=False).reset_index(drop=True)

    ladder = pd.concat([chall_df, gm_df, master_df]).reset_index(drop=True)[:top]
    ladder = ladder.drop(columns='rank').reset_index(drop=False).rename(columns={'index':'rank'})
    ladder['rank'] += 1
    
    return ladder

def get_match_history(puuid=None, region=None, start=0, count=20):
    
    root_url = f'https://{region}.api.riotgames.com'
    endpoint_url = f'/lol/match/v5/matches/by-puuid/{puuid}/ids'
    query_params = f'?start={start}&count={count}'

    response = requests.get(root_url + endpoint_url + query_params + f'&api_key={api_key}')

    return response.json()

def get_match_data_from_id(matchId=None, region=None):
    root_url = f'https://{region}.api.riotgames.com'
    endpoint_url = f'/lol/match/v5/matches/{matchId}'

    response = requests.get(root_url + endpoint_url + f'?api_key={api_key}')

    return response.json()

def get_match_history(puuid=None, region=None, start=0, count=20, queue=None):
    
    root_url = f'https://{region}.api.riotgames.com'
    endpoint_url = f'/lol/match/v5/matches/by-puuid/{puuid}/ids'
    query_params = f'?start={start}&count={count}'
    
    if queue is not None:
        query_params += f'&queue={queue}'

    response = requests.get(root_url + endpoint_url + query_params + f'&api_key={api_key}')

    return response.json()

def get_match_data_from_id(matchId=None, region=None):
    
    root_url = f'https://{region}.api.riotgames.com'
    endpoint_url = f'/lol/match/v5/matches/{matchId}'
    print(root_url + endpoint_url + f'?api_key={api_key}')
    response = requests.get(root_url + endpoint_url + f'?api_key={api_key}')

    return response.json()

def process_match_json(match_json, puuid):
    ##Architecture
    metadata = match_json['metadata']
    info = match_json['info']
    players = info['participants']
    participants = metadata['participants']
    teams = info['teams']
    player = players[participants.index(puuid)]
    perks = player['perks']
    stats = perks['statPerks']
    styles = perks['styles']
    

    primary = styles[0]
    secondary = styles[1]
    
    side_dict = {
        100: 'blue',
        200: 'red'
    }

    #information
    match_id = metadata['matchId']

    game_creation = info['gameCreation']
    game_duration = info['gameDuration']
    game_start_timestamp = info['gameStartTimestamp']
    game_end_timestamp = info['gameEndTimestamp']
    patch = info['gameVersion']
    game_mode = info['gameMode']
    game_type = info['gameType']
    queue_id = info['queueId']

    riot_id = player['riotIdGameName']
    riot_tag = player['riotIdTagline']
    summoner_id = player['summonerId']
    summoner_name = player['summonerName']
    
    side = side_dict[player['teamId']]
    win = player['win']

    champ_id = player['championId']
    champ_transform = player['championTransform']
    champ_level = player['champLevel']
    team_position = player['teamPosition']
    
    gold_earned = player['goldEarned']
    gold_spent = player['goldSpent']
    neutral_minions_killed = player['neutralMinionsKilled']
    total_minions_killed = player['totalMinionsKilled']
    total_ally_jungle_minions_killed = player['totalAllyJungleMinionsKilled']
    total_enemy_jungle_minions_killed = player['totalEnemyJungleMinionsKilled']
    
    kills = player['kills']
    deaths = player['deaths']
    assists = player['assists']
    first_blood = player['firstBloodKill']
    
    total_damage_dealt = player['totalDamageDealtToChampions']
    total_damage_shielded_on_teammates = player['totalDamageShieldedOnTeammates']
    total_damage_taken = player['totalDamageTaken']
    total_heal = player['totalHeal']
    total_heals_on_teammates = player['totalHealsOnTeammates']
    total_time_cc_dealt = player['totalTimeCCDealt']
    
    barons_killed = player['baronKills']
    turrets_lost = player['turretsLost']
    objectives_stolen = player['objectivesStolen']
    objectives_stolen_assist = player['objectivesStolenAssists']

    early_surrender = player['gameEndedInEarlySurrender']
    surrender = player['gameEndedInSurrender']

    item0 = player['item0']
    item1 = player['item1']
    item2 = player['item2']
    item3 = player['item3']
    item4 = player['item4']
    item5 = player['item5']
    item6 = player['item6']
    
    summoner1_id = player['summoner1Id']
    summoner2_id = player['summoner2Id']
    
    wards_placed = player['wardsPlaced']
    wards_killed = player['wardsKilled']
    vision_score = player['visionScore']
    detector_wards_placed = player['detectorWardsPlaced']

    defense = stats['defense']
    flex = stats['flex']
    offense = stats['offense']

    ##what is this
    role = player['role']
    vision_wards_bought_in_game = player['visionWardsBoughtInGame']

    primary_style = primary['style']
    secondary_style = secondary['style']

    primary_keystone = primary['selections'][0]['perk']
    primary_perk_1 = primary['selections'][1]['perk']
    primary_perk_2 = primary['selections'][2]['perk']
    primary_perk_3 = primary['selections'][3]['perk']

    secondary_perk_1 = secondary['selections'][0]['perk']
    secondary_perk_2 = secondary['selections'][1]['perk']

    #need to implement bans

    matchDF = pd.DataFrame({
        'match_id': [match_id],
        'game_creation': [game_creation],
        'game_duration': [game_duration],
        'game_start_timestamp': [game_start_timestamp],
        'game_end_timestamp': [game_end_timestamp],
        'patch': [patch],
        'game_mode': [game_mode],
        'game_type': [game_type],
        'queue_id': [queue_id],
        'riot_id': [riot_id],
        'riot_tag': [riot_tag],
        'summoner_id': [summoner_id],
        'summoner_name': [summoner_name],
        'side': [side],
        'win': [win],
        'puuid': [puuid],
        'champion_id': [champ_id],
        'champion_transform': [champ_transform],
        'champion_level': [champ_level],
        'team_position': [team_position],
        'item0': [item0],
        'item1': [item1],
        'item2': [item2],
        'item3': [item3],
        'item4': [item4],
        'item5': [item5],
        'item6': [item6],
        'summoner1_id': [summoner1_id],
        'summoner2_id': [summoner2_id],
        'neutral_minions_killed': [neutral_minions_killed],
        'total_minions_killed': [total_minions_killed],
        'total_ally_jungle_minions_killed': [total_ally_jungle_minions_killed],
        'total_enemy_jungle_minions_killed': [total_enemy_jungle_minions_killed],
        'objectives_stolen': [objectives_stolen],
        'objectives_stolen_assist': [objectives_stolen_assist],
        'kills': [kills],
        'deaths': [deaths],
        'assists': [assists],
        'gold_earned': [gold_earned],
        'gold_spent': [gold_spent],
        'total_damage_dealt': [total_damage_dealt],
        'total_damage_taken': [total_damage_taken],
        'total_heal': [total_heal],
        'total_heals_on_teammates': [total_heals_on_teammates],
        'total_time_cc_dealt': [total_time_cc_dealt],
        'barons_killed': [barons_killed],
        'first_blood': [first_blood],
        'early_surrender': [early_surrender],
        'surrender': [surrender],
        'wards_placed': [wards_placed],
        'wards_killed': [wards_killed],
        'vision_score': [vision_score],
        'detector_wards_placed': [detector_wards_placed],
        'role': [role],
        'vision_wards_bought_in_game': [vision_wards_bought_in_game],
        'primary_style': [primary_style],
        'secondary_style': [secondary_style],
        'primary_keystone': [primary_keystone],
        'primary_perk_1': [primary_perk_1],
        'primary_perk_2': [primary_perk_2],
        'primary_perk_3': [primary_perk_3],
        'secondary_perk_1': [secondary_perk_1],
        'secondary_perk_2': [secondary_perk_2],
        'defense': [defense],
        'flex': [flex],
        'offense': [offense],
        'turrets_lost': [turrets_lost],
        'total_damage_shielded_on_teammates': [total_damage_shielded_on_teammates],
    })
    return matchDF

RGAPI-a76bbb3e-4659-4021-b035-f0845041d3ed


In [3]:
# Get PUUID
print("Gingii puuid >>", get_puuid(gameName='Robb Stark', tagline='444', api_key=api_key))
print("Moldrn puuid >>", get_puuid(gameName='2018 Ame', tagline='PSG', api_key=api_key))
print("Frozen puuid >>", get_puuid(gameName='TwInkMage', tagline='QTK', api_key=api_key))
print("Messefy puuid >>", get_puuid(gameName='messefy', tagline='6969', api_key=api_key))
print("VaxerDJ puuid >>", get_puuid(gameName='I love Alt girls', tagline='Love', api_key=api_key))

Gingii puuid >> -RPH9IeTdhsXIIlsN4nzUMr73O04IqK_eeLiyKGUtd2cO5kM9jHvBZgeAaU-TATkh_EAjAo-qVHDLA
Moldrn puuid >> JDyvcgSiNxveLcmmbonQfiX15t_85UKaRjWAerymavmnWtYXPPqW328TJgzHP9kLVKBpMEHcLMs06A
Frozen puuid >> kjjaoPkm0xC9OOmydAb6E_ppk1Xyt_BxXdiR_VBMHzQB0IMhH3uHhQHamFUKZe-nItJE3OseV_Qj0g
Messefy puuid >> 2SesTfJYPDy2DWBeZhvgc2JoE2VxpbzMnGHxH-88JBKm7uhscdPmq0zvBHN7fUTsekdiQOL11jlIpA
VaxerDJ puuid >> 5-yu7_qiMBXJER-q-X08761VGYmPNn4ibZL46avwbBdtpuM8cNm6gKZWY6BbcBTn4lbRS5s2S6SYDg


In [4]:
gingii_puuid = '-RPH9IeTdhsXIIlsN4nzUMr73O04IqK_eeLiyKGUtd2cO5kM9jHvBZgeAaU-TATkh_EAjAo-qVHDLA'
moldrn_puuid = 'JDyvcgSiNxveLcmmbonQfiX15t_85UKaRjWAerymavmnWtYXPPqW328TJgzHP9kLVKBpMEHcLMs06A'
frozen_puuid = 'kjjaoPkm0xC9OOmydAb6E_ppk1Xyt_BxXdiR_VBMHzQB0IMhH3uHhQHamFUKZe-nItJE3OseV_Qj0g'
messefy_puuid = '2SesTfJYPDy2DWBeZhvgc2JoE2VxpbzMnGHxH-88JBKm7uhscdPmq0zvBHN7fUTsekdiQOL11jlIpA'
vaxerdj_puuid = '5-yu7_qiMBXJER-q-X08761VGYmPNn4ibZL46avwbBdtpuM8cNm6gKZWY6BbcBTn4lbRS5s2S6SYDg'

In [5]:
#GET GINGII'S LAST [COUNT] MATCHES
match_ids_gingii = get_match_history(puuid='-RPH9IeTdhsXIIlsN4nzUMr73O04IqK_eeLiyKGUtd2cO5kM9jHvBZgeAaU-TATkh_EAjAo-qVHDLA', region='europe', queue=420)
df_gingii = pd.DataFrame()
for match_id in match_ids_gingii:
    game = get_match_data_from_id(matchId=match_id, region='europe')
    matchDF = process_match_json(game, puuid='-RPH9IeTdhsXIIlsN4nzUMr73O04IqK_eeLiyKGUtd2cO5kM9jHvBZgeAaU-TATkh_EAjAo-qVHDLA')

    df_gingii = pd.concat([df_gingii, matchDF])

https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7579600820?api_key=RGAPI-a76bbb3e-4659-4021-b035-f0845041d3ed
https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7579562715?api_key=RGAPI-a76bbb3e-4659-4021-b035-f0845041d3ed
https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7579404048?api_key=RGAPI-a76bbb3e-4659-4021-b035-f0845041d3ed
https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7579351230?api_key=RGAPI-a76bbb3e-4659-4021-b035-f0845041d3ed
https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7579328232?api_key=RGAPI-a76bbb3e-4659-4021-b035-f0845041d3ed
https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7578856799?api_key=RGAPI-a76bbb3e-4659-4021-b035-f0845041d3ed
https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7578778887?api_key=RGAPI-a76bbb3e-4659-4021-b035-f0845041d3ed
https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7578715760?api_key=RGAPI-a76bbb3e-4659-4021-b035-f0845041d3ed
https://europe.api.riotgames.com

In [6]:
#GET MOLDRN'S LAST [COUNT] MATCHES
match_ids_moldrn = get_match_history(puuid='JDyvcgSiNxveLcmmbonQfiX15t_85UKaRjWAerymavmnWtYXPPqW328TJgzHP9kLVKBpMEHcLMs06A', region='europe', queue=420)
df_moldrn = pd.DataFrame()
for match_id in match_ids_moldrn:
    game = get_match_data_from_id(matchId=match_id, region='europe')
    matchDF = process_match_json(game, puuid='JDyvcgSiNxveLcmmbonQfiX15t_85UKaRjWAerymavmnWtYXPPqW328TJgzHP9kLVKBpMEHcLMs06A')

    df_moldrn = pd.concat([df_moldrn, matchDF])

https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7579589928?api_key=RGAPI-a76bbb3e-4659-4021-b035-f0845041d3ed
https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7579542129?api_key=RGAPI-a76bbb3e-4659-4021-b035-f0845041d3ed
https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7578876927?api_key=RGAPI-a76bbb3e-4659-4021-b035-f0845041d3ed
https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7578838527?api_key=RGAPI-a76bbb3e-4659-4021-b035-f0845041d3ed
https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7578798893?api_key=RGAPI-a76bbb3e-4659-4021-b035-f0845041d3ed
https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7578747974?api_key=RGAPI-a76bbb3e-4659-4021-b035-f0845041d3ed
https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7578707008?api_key=RGAPI-a76bbb3e-4659-4021-b035-f0845041d3ed
https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7578165658?api_key=RGAPI-a76bbb3e-4659-4021-b035-f0845041d3ed
https://europe.api.riotgames.com

In [ ]:
#GET FROZEN'S LAST 20 MATCHES
match_ids_frozen = get_match_history(puuid='kjjaoPkm0xC9OOmydAb6E_ppk1Xyt_BxXdiR_VBMHzQB0IMhH3uHhQHamFUKZe-nItJE3OseV_Qj0g', region='europe', queue=420)
df_frozen = pd.DataFrame()
for match_id in match_ids_frozen:
    game = get_match_data_from_id(matchId=match_id, region='europe')
    matchDF = process_match_json(game, puuid='kjjaoPkm0xC9OOmydAb6E_ppk1Xyt_BxXdiR_VBMHzQB0IMhH3uHhQHamFUKZe-nItJE3OseV_Qj0g')

    df_frozen = pd.concat([df_frozen, matchDF])

In [ ]:
#GET MESSEFY'S LAST 20 MATCHES
match_ids_messefy = get_match_history(puuid='2SesTfJYPDy2DWBeZhvgc2JoE2VxpbzMnGHxH-88JBKm7uhscdPmq0zvBHN7fUTsekdiQOL11jlIpA', region='europe', queue=420)
df_messefy = pd.DataFrame()
for match_id in match_ids_messefy:
    game = get_match_data_from_id(matchId=match_id, region='europe')
    matchDF = process_match_json(game, puuid='2SesTfJYPDy2DWBeZhvgc2JoE2VxpbzMnGHxH-88JBKm7uhscdPmq0zvBHN7fUTsekdiQOL11jlIpA')

    df_messefy = pd.concat([df_messefy, matchDF])
    

In [ ]:
#GET VAXERDJ'S LAST 20 MATCHES
match_ids_vaxerdj = get_match_history(puuid='5-yu7_qiMBXJER-q-X08761VGYmPNn4ibZL46avwbBdtpuM8cNm6gKZWY6BbcBTn4lbRS5s2S6SYDg', region='europe', queue=420)
df_vaxerdj = pd.DataFrame()
for match_id in match_ids_vaxerdj:
    game = get_match_data_from_id(matchId=match_id, region='europe')
    matchDF = process_match_json(game, puuid='5-yu7_qiMBXJER-q-X08761VGYmPNn4ibZL46avwbBdtpuM8cNm6gKZWY6BbcBTn4lbRS5s2S6SYDg')

    df_vaxerdj = pd.concat([df_vaxerdj, matchDF])

In [7]:
#PRINT GINGII'S LAST 20 MATCHES
df_gingii

,match_id,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,game_mode,game_type,queue_id,riot_id,...,primary_perk_1,primary_perk_2,primary_perk_3,secondary_perk_1,secondary_perk_2,defense,flex,offense,turrets_lost,total_damage_shielded_on_teammates
0,EUW1_7579600820,1761327251055,1772,1761327310008,1761329082388,15.21.721.4012,CLASSIC,MATCHED_GAME,420,Robb Stark,...,9111,9105,8299,8453,8444,5001,5008,5008,7,0
0,EUW1_7579562715,1761325411631,1478,1761325437107,1761326915116,15.21.721.4012,CLASSIC,MATCHED_GAME,420,Robb Stark,...,8224,8234,8236,8451,8444,5001,5008,5008,9,0
0,EUW1_7579404048,1761318124759,928,1761318143502,1761319071760,15.21.721.4012,CLASSIC,MATCHED_GAME,420,Robb Stark,...,9111,9105,8299,8453,8444,5011,5008,5008,3,0
0,EUW1_7579351230,1761315592663,1633,1761315625676,1761317258440,15.21.721.4012,CLASSIC,MATCHED_GAME,420,Robb Stark,...,8226,8210,8237,8429,8451,5001,5001,5005,2,0
0,EUW1_7579328232,1761314249920,1029,1761314272225,1761315301371,15.21.721.4012,CLASSIC,MATCHED_GAME,420,Robb Stark,...,8401,8444,8451,8304,8345,5011,5008,5008,4,0
0,EUW1_7578856799,1761256859115,1186,1761256920900,1761258106860,15.21.720.925,CLASSIC,MATCHED_GAME,420,Robb Stark,...,9111,9104,8299,8224,8234,5011,5008,5005,2,0
0,EUW1_7578778887,1761253702348,2023,1761253729258,1761255752586,15.21.720.925,CLASSIC,MATCHED_GAME,420,Robb Stark,...,8401,8473,8451,8304,8345,5011,5008,5008,9,0
0,EUW1_7578715760,1761251443965,1683,1761251466716,1761253150093,15.21.720.925,CLASSIC,MATCHED_GAME,420,Robb Stark,...,8401,8473,8451,8304,8345,5011,5008,5008,4,0
0,EUW1_7578165658,1761228892008,1600,1761228917669,1761230518031,15.21.720.925,CLASSIC,MATCHED_GAME,420,Robb Stark,...,8224,8234,8236,8451,8444,5011,5008,5008,10,0
0,EUW1_7578118059,1761226456059,1679,1761226480861,1761228160029,15.21.720.925,CLASSIC,MATCHED_GAME,420,Robb Stark,...,9111,9105,8299,8453,8444,5011,5008,5008,5,0


In [ ]:
#PRINT MOLDRN'S LAST 20 MATCHES
df_moldrn

In [ ]:
#PRINT FROZEN'S LAST 20 MATCHES
df_frozen

In [ ]:
#PRINT MESSEFY'S LAST 20 MATCHES
df_messefy

In [ ]:
#DICTIONARY
import re 
# Base URL for all resources
base_url = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/'

# List of JSON URLs (only those ending in .json)
json_urls = {
    'perks': base_url + 'perks.json',
    'perkstyles': base_url + 'perkstyles.json',
    'items': base_url + 'items.json',
    'queues': base_url + 'queues.json',
    'summoner_spells': base_url + 'summoner-spells.json',
    # Add this for champion IDs/names/icons (it's a summary JSON with all champions)
    'champion_summary': base_url + 'champion-summary.json'
}

# Directory URLs (for images; we'll handle via JSON paths instead)
image_dirs = {
    'champ_icons': base_url + 'champion-icons/',
    'champions': base_url + 'champions/',  # For individual champion JSONs, not images; use IDs from summary to fetch specific if needed
    'perk_styles_img': base_url + 'perk-images/styles/',
    'perk_stat_img': base_url + 'perk-images/statmods/'
}

# Fetch and process only JSON URLs
data_jsons = {}
for name, url in json_urls.items():
    if url.endswith('.json'):
        response = requests.get(url)
        if response.status_code == 200:
            data_jsons[name] = response.json()
        else:
            print(f"Failed to fetch {name} from {url}: {response.status_code}")

# Your json_extract function (unchanged)
def json_extract(obj, key):
    arr = []
    
    def extract(obj, arr, key):
        if isinstance(obj, dict):
            for k, v in obj.items():
                if k == key:
                    arr.append(v)
                elif isinstance(v, (dict, list)):
                    extract(v, arr, key)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
                
        return arr
    
    values = extract(obj, arr, key)
    return values

# Create dictionaries for JSONs that have extractable IDs/names (expand as needed)
# Example for perks
if 'perks' in data_jsons:
    perk_ids = json_extract(data_jsons['perks'], 'id')
    perk_names = json_extract(data_jsons['perks'], 'name')
    perk_dict = dict(map(lambda i, j: (int(i), j), perk_ids, perk_names))

# Example for perkstyles (note: IDs/names are under 'styles' key, but json_extract finds them recursively)
if 'perkstyles' in data_jsons:
    perk_styles_ids = json_extract(data_jsons['perkstyles'], 'id')
    perk_styles_names = json_extract(data_jsons['perkstyles'], 'name')
    perk_styles_dict = dict(map(lambda i, j: (int(i), j), perk_styles_ids, perk_styles_names))

# For items (list of dicts with 'id' and 'name')
if 'items' in data_jsons:
    item_dict = {entry['id']: entry['name'] for entry in data_jsons['items'] if 'id' in entry and 'name' in entry}

# For champions (list of dicts with 'id' and 'name')
if 'champion_summary' in data_jsons:
    champ_dict = {entry['id']: entry['name'] for entry in data_jsons['champion_summary'] if 'id' in entry and 'name' in entry}

# For summoner spells (list of dicts with 'id' and 'name')
if 'summoner_spells' in data_jsons:
    summoner_dict = {entry['id']: entry['name'] for entry in data_jsons['summoner_spells'] if 'id' in entry and 'name' in entry}

# Similarly for items, summoner_spells, etc. (structures are similar: list of dicts with 'id' and 'name')
if 'items' in data_jsons:
    item_ids = json_extract(data_jsons['items'], 'id')
    item_names = json_extract(data_jsons['items'], 'name')
    item_dict = dict(map(lambda i, j: (int(i), j), item_ids, item_names))

# For queues, it's a dict of str(queueId): obj with 'description', etc. – adjust extraction if needed
if 'queues' in data_jsons:
    queue_dict = {entry['id']: entry['description'] for entry in data_jsons['queues'] if 'id' in entry and 'description' in entry}

# For champion_summary (IDs/names/icons)
if 'champion_summary' in data_jsons:
    champ_ids = json_extract(data_jsons['champion_summary'], 'id')
    champ_names = json_extract(data_jsons['champion_summary'], 'name')
    champ_dict = dict(map(lambda i, j: (int(i), j), champ_ids, champ_names))

# Handling images (from JSON iconPaths instead of directories)
# Base for constructing full image URLs
image_base = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default'

# For champion icons (from champion-summary.json)
champ_icon_dict = {}  # id: full_image_url
if 'champion_summary' in data_jsons:
    for champ in data_jsons['champion_summary']:
        if 'id' in champ and 'squarePortraitPath' in champ:
            champ_id = champ['id']
            rel_path = champ['squarePortraitPath']
            full_url = image_base + rel_path
            champ_icon_dict[champ_id] = full_url

# For perk images (including keystones and regular perks; from perks.json)
perk_icon_dict = {}  # id: full_image_url
if 'perks' in data_jsons:
    for perk in data_jsons['perks']:
        if 'id' in perk and 'iconPath' in perk:
            perk_id = perk['id']
            rel_path = perk['iconPath']
            full_url = image_base + rel_path
            perk_icon_dict[perk_id] = full_url

# For stat mod images (stat shards are special perks with IDs 5001-5013; also from perks.json)
stat_icon_dict = {}  # id: full_image_url
if 'perks' in data_jsons:
    for perk in data_jsons['perks']:
        if 'id' in perk and str(perk['id']).startswith('50') and 'iconPath' in perk:  # Filter for 500x IDs
            stat_id = perk['id']
            rel_path = perk['iconPath']
            full_url = image_base + rel_path
            stat_icon_dict[stat_id] = full_url

# For perk style images (from perkstyles.json)
style_icon_dict = {}  # id: full_image_url
if 'perkstyles' in data_jsons:
    for style in data_jsons['perkstyles']['styles']:
        if 'id' in style and 'iconPath' in style:
            style_id = style['id']
            rel_path = style['iconPath']
            full_url = image_base + rel_path
            style_icon_dict[style_id] = full_url

# If you REALLY need to list files from an image directory (e.g., if JSON paths are insufficient), here's how:
# Example for champ_icons directory (fetches HTML listing, extracts .png links with regex)
def list_dir_files(dir_url):
    response = requests.get(dir_url)
    if response.status_code == 200:
        text = response.text
        # Extract <a href="...png"> links (assumes Apache-style HTML listing)
        files = re.findall(r'<a href="([^"]+\.png)">', text)
        return files
    else:
        print(f"Failed to list {dir_url}: {response.status_code}")
        return []

In [ ]:
# Inverted dicts for reversal (name back to ID)
inverted_perk_dict = {v: k for k, v in perk_dict.items()}
inverted_perk_styles_dict = {v: k for k, v in perk_styles_dict.items()}
inverted_item_dict = {v: k for k, v in item_dict.items()}
inverted_champ_dict = {v: k for k, v in champ_dict.items()}
inverted_summoner_dict = {v: k for k, v in summoner_dict.items()}
inverted_champ_dict = {v: k for k, v in champ_dict.items()}

df_gingii['summoner1_id'] = df_gingii['summoner1_id'].replace(inverted_champ_dict)
df_gingii['summoner2_id'] = df_gingii['summoner2_id'].replace(inverted_champ_dict)

df_moldrn['summoner1_id'] = df_moldrn['summoner1_id'].replace(inverted_champ_dict)
df_moldrn['summoner2_id'] = df_moldrn['summoner2_id'].replace(inverted_champ_dict)

df_frozen['summoner1_id'] = df_frozen['summoner1_id'].replace(inverted_champ_dict)
df_frozen['summoner2_id'] = df_frozen['summoner2_id'].replace(inverted_champ_dict)

df_messefy['summoner1_id'] = df_messefy['summoner1_id'].replace(inverted_champ_dict)
df_messefy['summoner2_id'] = df_messefy['summoner2_id'].replace(inverted_champ_dict)

df_vaxerdj['summoner1_id'] = df_vaxerdj['summoner1_id'].replace(inverted_champ_dict)
df_vaxerdj['summoner2_id'] = df_vaxerdj['summoner2_id'].replace(inverted_champ_dict)
# Then apply the column-specific replace above

# Expanded list of all numerical/stat columns to protect/reverse (add any others from your df if missing)
protected_cols = [
    'game_creation', 'game_duration', 'game_start_timestamp', 'game_end_timestamp',
    'queue_id', 'champion_transform', 'champion_level', 'neutral_minions_killed',
    'total_minions_killed', 'total_ally_jungle_minions_killed', 'total_enemy_jungle_minions_killed',
    'objectives_stolen', 'objectives_stolen_assist', 'kills', 'deaths', 'assists',
    'gold_earned', 'gold_spent', 'total_damage_dealt', 'total_damage_taken',
    'total_heal', 'total_heals_on_teammates', 'total_time_cc_dealt', 'barons_killed',
    'wards_placed', 'wards_killed', 'vision_score', 'detector_wards_placed',
    'vision_wards_bought_in_game', 'turrets_lost', 'total_damage_shielded_on_teammates'
]

# Reverse any bad mappings on protected columns
for col in protected_cols:
    if col in df_gingii.columns:
        df_gingii[col] = df_gingii[col].replace(inverted_perk_dict).replace(inverted_perk_styles_dict)\
                         .replace(inverted_item_dict).replace(inverted_champ_dict)\
                         .replace(inverted_summoner_dict)
        # Force numeric type after reversal (handles any lingering strings)
        df_gingii[col] = pd.to_numeric(df_gingii[col], errors='coerce').fillna(df_gingii[col])

for col in protected_cols:
    if col in df_moldrn.columns:
        df_moldrn[col] = df_moldrn[col].replace(inverted_perk_dict).replace(inverted_perk_styles_dict)\
                         .replace(inverted_item_dict).replace(inverted_champ_dict)\
                         .replace(inverted_summoner_dict)
        # Force numeric type after reversal (handles any lingering strings)
        df_moldrn[col] = pd.to_numeric(df_moldrn[col], errors='coerce').fillna(df_moldrn[col])
        
for col in protected_cols:
    if col in df_frozen.columns:
        df_frozen[col] = df_frozen[col].replace(inverted_perk_dict).replace(inverted_perk_styles_dict)\
                         .replace(inverted_item_dict).replace(inverted_champ_dict)\
                         .replace(inverted_summoner_dict)
        # Force numeric type after reversal (handles any lingering strings)
        df_frozen[col] = pd.to_numeric(df_frozen[col], errors='coerce').fillna(df_frozen[col])

for col in protected_cols:
    if col in df_messefy.columns:
        df_messefy[col] = df_messefy[col].replace(inverted_perk_dict).replace(inverted_perk_styles_dict)\
                         .replace(inverted_item_dict).replace(inverted_champ_dict)\
                         .replace(inverted_summoner_dict)
        # Force numeric type after reversal (handles any lingering strings)
        df_messefy[col] = pd.to_numeric(df_messefy[col], errors='coerce').fillna(df_messefy[col])
    
for col in protected_cols:
    if col in df_vaxerdj.columns:
        df_vaxerdj[col] = df_vaxerdj[col].replace(inverted_perk_dict).replace(inverted_perk_styles_dict)\
                         .replace(inverted_item_dict).replace(inverted_champ_dict)\
                         .replace(inverted_summoner_dict)
        # Force numeric type after reversal (handles any lingering strings)
        df_vaxerdj[col] = pd.to_numeric(df_vaxerdj[col], errors='coerce').fillna(df_vaxerdj[col])

In [ ]:
#DICT REPLACEMENTS FOR GINGII
pd.options.display.max_columns = 100

df_gingii = df_gingii.replace({
    
    'primary_style': perk_styles_dict,
    'secondary_style': perk_styles_dict,
    
    'primary_keystone': perk_dict,
    'primary_perk_1': perk_dict,
    'primary_perk_2': perk_dict,
    'primary_perk_3': perk_dict,
    'secondary_perk_1': perk_dict,
    'secondary_perk_2': perk_dict,
    
    'defense': perk_dict,
    'flex': perk_dict,
    'offense': perk_dict,
    
    'champion_id': champ_dict,

    'summoner1_id': summoner_dict,
    'summoner2_id': summoner_dict,
    
    'item0': item_dict,
    'item1': item_dict,
    'item2': item_dict,
    'item3': item_dict,
    'item4': item_dict,
    'item5': item_dict,
    'item6': item_dict
})

df_gingii


In [ ]:
#PRINT GINGII'S LAST 20 MATCHES ON THE GOOGLE SPREADSHEET [sheet1]
wks = sheet[0]

wks.set_dataframe(df_gingii, start='A1', copy_head=True, fit=True)

In [ ]:
#DICT REPLACEMENTS FOR MOLDRN
pd.options.display.max_columns = 100

df_moldrn = df_moldrn.replace({

    'primary_style': perk_styles_dict,
    'secondary_style': perk_styles_dict,
    
    'primary_keystone': perk_dict,
    'primary_perk_1': perk_dict,
    'primary_perk_2': perk_dict,
    'primary_perk_3': perk_dict,
    'secondary_perk_1': perk_dict,
    'secondary_perk_2': perk_dict,
    
    'defense': perk_dict,
    'flex': perk_dict,
    'offense': perk_dict,
    
    'champion_id': champ_dict,
    
    'summoner1_id': summoner_dict,
    'summoner2_id': summoner_dict,
    
    'item0': item_dict,
    'item1': item_dict,
    'item2': item_dict,
    'item3': item_dict,
    'item4': item_dict,
    'item5': item_dict,
    'item6': item_dict
})

df_moldrn

In [ ]:
#PRINT MOLDRN'S LAST 20 MATCHES ON THE GOOGLE SPREADSHEET [sheet2]
wks = sheet[1]


wks.set_dataframe(df_moldrn, start='A1', copy_head=True, fit=True)

In [ ]:
#DICT REPLACEMENTS FOR FROZEN
pd.options.display.max_columns = 100

df_frozen = df_frozen.replace({
    
    'primary_style': perk_styles_dict,
    'secondary_style': perk_styles_dict,

    'primary_keystone': perk_dict,
    'primary_perk_1': perk_dict,
    'primary_perk_2': perk_dict,
    'primary_perk_3': perk_dict,
    'secondary_perk_1': perk_dict,
    'secondary_perk_2': perk_dict,
    
    'defense': perk_dict,
    'flex': perk_dict,
    'offense': perk_dict,
    
    
    'champion_id': champ_dict,
    
    'summoner1_id': summoner_dict,
    'summoner2_id': summoner_dict,
    
    'item0': item_dict,
    'item1': item_dict,
    'item2': item_dict,
    'item3': item_dict,
    'item4': item_dict,
    'item5': item_dict,
    'item6': item_dict
})

df_frozen

In [ ]:
#PRINT FROZEN'S LAST 20 MATCHES ON THE GOOGLE SPREADSHEET [sheet3]
wks = sheet[2]

# Set the DataFrame to the worksheet, starting at row 1, column 1 (A1), with headers
wks.set_dataframe(df_frozen, start='A1', copy_head=True, fit=True)

In [ ]:
#DICT REPLACEMENTS FOR MESSEFY
pd.options.display.max_columns = 100

df_messefy = df_messefy.replace({
    
    'primary_style': perk_styles_dict,
    'secondary_style': perk_styles_dict,
    
    'primary_keystone': perk_dict,
    'primary_perk_1': perk_dict,
    'primary_perk_2': perk_dict,
    'primary_perk_3': perk_dict,
    'secondary_perk_1': perk_dict,
    'secondary_perk_2': perk_dict,
    
    'defense': perk_dict,
    'flex': perk_dict,
    'offense': perk_dict,
    
    'champion_id': champ_dict,
    
    'summoner1_id': summoner_dict,
    'summoner2_id': summoner_dict,
    
    'item0': item_dict,
    'item1': item_dict,
    'item2': item_dict,
    'item3': item_dict,
    'item4': item_dict,
    'item5': item_dict,
    'item6': item_dict
})

df_messefy

In [ ]:
#PRINT MESSEFY'S LAST 20 MATCHES ON THE GOOGLE SPREADSHEET [sheet4]
wks = sheet[3]


wks.set_dataframe(df_messefy, start='A1', copy_head=True, fit=True)

In [ ]:
#DICT REPLACEMENTS FOR VAXERDJ
pd.options.display.max_columns = 100

df_vaxerdj = df_vaxerdj.replace({
    
    'primary_style': perk_styles_dict,
    'secondary_style': perk_styles_dict,
    
    'primary_keystone': perk_dict,
    'primary_perk_1': perk_dict,
    'primary_perk_2': perk_dict,
    'primary_perk_3': perk_dict,
    'secondary_perk_1': perk_dict,
    'secondary_perk_2': perk_dict,
    
    'defense': perk_dict,
    'flex': perk_dict,
    'offense': perk_dict,
    
    'champion_id': champ_dict,
    
    'summoner1_id': summoner_dict,
    'summoner2_id': summoner_dict,
    
    'item0': item_dict,
    'item1': item_dict,
    'item2': item_dict,
    'item3': item_dict,
    'item4': item_dict,
    'item5': item_dict,
    'item6': item_dict
})

df_vaxerdj

In [ ]:
#PRINT VAXERDJ'S LAST 20 MATCHES ON THE GOOGLE SPREADSHEET [sheet5]
wks = sheet[4]

wks.set_dataframe(df_vaxerdj, start='A1', copy_head=True, fit=True)

In [ ]:
def get_rank(puuid=None, region='euw1'):
    root_url = f'https://{region}.api.riotgames.com'
    endpoint_url = f'/lol/league/v4/entries/by-puuid/{puuid}'

    response = requests.get(root_url + endpoint_url + f'?api_key={api_key}')
    
    if response.status_code != 200:
        raise ValueError(f"Failed to get rank data: {response.status_code} - {response.text}")
    
    return response.json()



In [ ]:
#GET GINGII RANKED INFO

def process_rank_gingii(gingii_rank_json):
    data = []
    
    for entry in gingii_rank_json:
        data.append({
            'leagueId': entry.get('leagueId'),
            'queueType': entry.get('queueType'),
            'tier': entry.get('tier'),
            'rank': entry.get('rank'),
            'puuid': entry.get('puuid'),
            'LP': entry.get('leaguePoints'),
            'wins': entry.get('wins'),
            'losses': entry.get('losses'),
        })
    
    gingii_rank_df = pd.DataFrame(data)
    return gingii_rank_df

In [ ]:
#get gingii ranked data
gingii_rank_json = get_rank(gingii_puuid)
gingii_rank_df = process_rank_gingii(gingii_rank_json)
gingii_rank_df


In [ ]:
#GET MOLDRN RANKED INFO

def process_rank_moldrn(moldrn_rank_json):
    data = []
    
    for entry in moldrn_rank_json:
        data.append({
            'leagueId': entry.get('leagueId'),
            'queueType': entry.get('queueType'),
            'tier': entry.get('tier'),
            'rank': entry.get('rank'),
            'puuid': entry.get('puuid'),
            'LP': entry.get('leaguePoints'),
            'wins': entry.get('wins'),
            'losses': entry.get('losses'),
        })
    
    moldrn_rank_df = pd.DataFrame(data)
    return moldrn_rank_df

In [ ]:
#get moldrn ranked data
moldrn_rank_json = get_rank(moldrn_puuid)
moldrn_rank_df = process_rank_moldrn(moldrn_rank_json)
moldrn_rank_df

In [ ]:
#GET Frozen RANKED INFO

def process_rank_frozen(frozen_rank_json):
    data = []
    
    for entry in frozen_rank_json:
        data.append({
            'leagueId': entry.get('leagueId'),
            'queueType': entry.get('queueType'),
            'tier': entry.get('tier'),
            'rank': entry.get('rank'),
            'puuid': entry.get('puuid'),
            'LP': entry.get('leaguePoints'),
            'wins': entry.get('wins'),
            'losses': entry.get('losses'),
        })
    
    frozen_rank_df = pd.DataFrame(data)
    return frozen_rank_df

In [ ]:
#get frozen ranked data
frozen_rank_json = get_rank(frozen_puuid)
frozen_rank_df = process_rank_frozen(frozen_rank_json)
frozen_rank_df

In [ ]:
#GET MESSEFY RANKED INFO

def process_rank_messefy(messefy_rank_json):
    data = []
    
    for entry in messefy_rank_json:
        data.append({
            'leagueId': entry.get('leagueId'),
            'queueType': entry.get('queueType'),
            'tier': entry.get('tier'),
            'rank': entry.get('rank'),
            'puuid': entry.get('puuid'),
            'LP': entry.get('leaguePoints'),
            'wins': entry.get('wins'),
            'losses': entry.get('losses'),
        })
    
    messefy_rank_df = pd.DataFrame(data)
    return messefy_rank_df

In [ ]:
#get messefy ranked data
messefy_rank_json = get_rank(messefy_puuid)
messefy_rank_df = process_rank_messefy(messefy_rank_json)
messefy_rank_df

In [ ]:
#GET VAXERDJ RANKED INFO

def process_rank_vaxerdj(vaxerdj_rank_json):
    data = []
    
    for entry in vaxerdj_rank_json:
        data.append({
            'leagueId': entry.get('leagueId'),
            'queueType': entry.get('queueType'),
            'tier': entry.get('tier'),
            'rank': entry.get('rank'),
            'puuid': entry.get('puuid'),
            'LP': entry.get('leaguePoints'),
            'wins': entry.get('wins'),
            'losses': entry.get('losses'),
        })
    
    vaxerdj_rank_df = pd.DataFrame(data)
    return vaxerdj_rank_df

In [ ]:
#get vaxerdj ranked data
vaxerdj_rank_json = get_rank(vaxerdj_puuid)
vaxerdj_rank_df = process_rank_vaxerdj(vaxerdj_rank_json)
vaxerdj_rank_df

In [ ]:
#RANKS INTO SHEETS
#************DONT TOUCH SHEET NUMBERS & DONT CHANGE SHEET ORDER IN THE SPREADSHEET********

#gingii sheet rank
wks = sheet[5]

wks.set_dataframe(gingii_rank_df, start='A1', copy_index=False, copy_head=True)

#moldrn sheet rank
wks = sheet[6]

wks.set_dataframe(moldrn_rank_df, start='A1', copy_index=False, copy_head=True)

#frozen sheet rank
wks = sheet[7]

wks.set_dataframe(frozen_rank_df, start='A1', copy_index=False, copy_head=True)

#messefy sheet rank
wks = sheet[7]

wks.set_dataframe(messefy_rank_df, start='A1', copy_index=False, copy_head=True)

#vaxerdj sheet rank
wks = sheet[8]

wks.set_dataframe(vaxerdj_rank_df, start='A1', copy_index=False, copy_head=True)
